In [3]:
!pip install transformers diffusers huggingface_hub datasets ipywidgets accelerate torchvision

In [1]:
import json
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from diffusers import StableDiffusionPipeline
from diffusers.models.modeling_outputs import Transformer2DModelOutput
from huggingface_hub import login
from PIL import Image

# Configurations
batch_size = 64
num_channels = 3  # RGB images have 3 channels
num_classes = 2
image_size = (128, 96)  # Resize to save memory
latent_dim = 128

# Paths to data
image_dir = '../FP/Data/HAM/ham_images'  # Directory containing images
csv_path = '../FP/Experiments/two_class_metadata.csv'  # Path to CSV file

# Load data from CSV
df = pd.read_csv(csv_path)

# Extract image IDs and classes
image_ids = df['image_id'].values
labels = df['class'].values

# Encode labels to integers then to one-hot
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
one_hot_labels = to_categorical(integer_encoded, num_classes=num_classes)

# Define function to get full image path
def get_image_path(image_id):
    return f"{image_dir}/{image_id}.jpg"

# Load and process images
def process_image(image_id):
    full_path = get_image_path(image_id)
    img = load_img(full_path, target_size=image_size, color_mode='rgb')  # Load as RGB
    img_array = img_to_array(img).astype("float32") / 255.0
    return img_array

# Create datasets
all_images = np.array([process_image(image_id) for image_id in image_ids])
all_labels = one_hot_labels

# Ensure the images have 4 dimensions (batch_size, height, width, channels)
all_images = np.reshape(all_images, (-1, image_size[0], image_size[1], num_channels))

# Create tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((all_images, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

# Print data shapes for confirmation
print(f"Shape of images: {all_images.shape}")
print(f"Shape of labels: {all_labels.shape}")

# Login to Hugging Face to access the model
login()  # You will be prompted to enter your Hugging Face API token

# Load the Stable Diffusion model from Hugging Face
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe.to("cuda")  # Use GPU for faster generation

# Function to generate synthetic images using existing dataset images as references
def generate_synthetic_images(images, labels, num_images):
    synthetic_images = []
    for i in range(num_images):
        label_text = "Benign" if np.argmax(labels[i]) == 0 else "Malignant"
        prompt = f"{label_text} skin lesion"

        # Use the existing image as a reference
        ref_image = Image.fromarray((images[i] * 255).astype(np.uint8))
        ref_image = ref_image.resize((512, 512))

        # Generate synthetic image based on prompt and reference image
        generated_image = pipe(prompt, ref_image).images[0]
        generated_image = generated_image.resize(image_size)  # Resize to the required image size
        generated_image = np.array(generated_image) / 255.0  # Normalize the image

        synthetic_images.append(generated_image)
    return np.array(synthetic_images)

# Example usage: Generate 10 synthetic images using reference images
num_synthetic_images = 10
synthetic_images = generate_synthetic_images(all_images[:num_synthetic_images], all_labels[:num_synthetic_images], num_synthetic_images)

# Convert synthetic images to a tf.data.Dataset
synthetic_dataset = tf.data.Dataset.from_tensor_slices((synthetic_images, all_labels[:num_synthetic_images]))
synthetic_dataset = synthetic_dataset.batch(batch_size)

# Function to save synthetic images
def save_synthetic_images(images, save_dir):
    import os
    from PIL import Image
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for i, img in enumerate(images):
        img = Image.fromarray((img * 255).astype(np.uint8))
        img.save(os.path.join(save_dir, f"synthetic_image_{i}.png"))

# Save generated synthetic images
save_synthetic_images(synthetic_images, "./synthetic_images")

# Verify the synthetic dataset
for images, labels in synthetic_dataset.take(1):
    print(f"Synthetic images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")


/Users/andreshofmann/.pyenv/versions/3.12.2/envs/.venv/lib/python3.12/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


Shape of images: (10015, 128, 96, 3)
Shape of labels: (10015, 2)


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:  43%|####3     | 1.48G/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled